In [1]:
import pandas as pd
from math import pi, sin, cos, exp, tan, acos, isclose

# Dicinoário de Dados
COEFICIENTE_BASAL    => KCB\
COEFICIENTE_CULTURA  => KC\
COEFICIENTE_EVAPO    => KE\
EVAPO_CULTURA        => ETC\
TRANSP_POTENCIAL     => TRP\
EVAPO                => E\
ARMAZENAMENTO_AGUA   => ARM\
Capacidade de Campo  => TAR\
ARM2                 => ARM2\
EVAPO_REFERENCIA     => ET0\
EVAPO_REAL           => ETR\
TR                   => TR\
TA                   => TA\
DIAJULIANO           => DIAJULIANO\
TARMEDIO             => TAR/2 \
Indice de Satisfação => ISNA\
PLUVIOSIDADE         => P\


In [2]:
#Termo Aerodinâmico    
def termoAeroDinamico(params, urx, urn, tx, tn):
    tm   = (tx + tn)/2
    #Pressão de saturação do vapor d'água do ar - Equação de Tetens [kPa]
    es   = (0.6108* exp(17.27*tx/(237.3+tx))+ 0.6108* exp(17.27*tn/(237.3+tn)))/2
    #Pressão real do vapor d'água do ar [kPa]   
    ea   = (urn * 0.6108* exp(17.27*tx/(237.3+tx))+ urx * 0.6108* exp(17.27*tn/(237.3+tn)))/ 200
    #Declividade da curva de pressão de saturação [kPa/oC]
    S    = 4098 * 0.6108* exp(17.27*tm/(237.3+tm))/((237.3 + tm)**2)
    #Pressão atmosférica [kPa]
    Patm = 101.3*((293-0.0065*params.Z)/(293))**5.26
    #Calor latente de evaporação [MJ/kg]
    lamb = 0.665E-3*Patm
    
    return S, tm, es, ea, lamb

#Termo Radioativo
def saldoDeRadiacao(params, doy, qg, tx, tn, ea):
    #Radiação solar extraterrestre

    #Correção relativa Terra-Sol
    dr   = 1 + 0.033 * cos(2*pi/365*doy)
    #Declinação Solar 
    decl = 0.409 * sin((2*pi/365*doy)-1.39)
    #Angulo Horário
    ws   = acos(-(params.FI*pi/180)*tan(decl))
    #Radiação Solar Extraterrestre
    Qo   = 37.568*dr*((ws*sin(params.FI*pi/180)*sin(decl))+(cos(params.FI*pi/180)*cos(decl)*sin(ws)))

    #Balanço de radiação
    #Ondas curtas
    
    #Radiação solar para dia de céu sem nebulosidade
    Qso = (0.75 + 2e-5*params.Z)*Qo
    Qoc = 0.77 * qg
    
    #Ondas longas
    Qol    = 4.903E-9*((((tx + 273.16)**4)+ ((tn + 273.16)**4))/2)*(0.34-0.14*ea**0.5)*(1.35*(qg/Qso)-0.35)

    return Qoc - Qol

#Transpiração Potencial
def transpiracaoPotencial(params, u2, urn, aETo_PM, t):
    if t < params.L_INI:
        Kcb = params.KCB_INI
    elif t < params.L_INI + params.L_CRES:
        Kcb = params.KCB_INI+(t-params.L_INI)/params.L_CRES*(params.KCB_MID-params.KCB_INI)
    elif t < params.L_INI + params.L_CRES + params.L_MID:
        Kcb = params.KCB_MID
    else:
        Kcb = params.KCB_MID+(t-(params.L_INI+params.L_CRES+params.L_MID))/params.L_FIM*(params.KCB_FIM-params.KCB_MID)
    
    h = max((Kcb/params.KCB_MID)*params.HX,params.H)
    kcx = max(1.2+(0.04*(u2-2)-0.004*(urn-45))*(params.H/3)**0.3,Kcb+0.05)

    return Kcb, kcx 

#Balanço de água no solo
def evaporacao(params, p, Kcb, kcx, ETo_PM, De_f, t):
    #Camada subsuperfícial - Evaporação
    fc = max(abs(((Kcb-params.KCB_INI)/(kcx-params.KCB_INI)))**(1 + 0.5*params.H),0.01)
    
    if p > 0: fw = 1
    else: fw = params.FW_INI
    
    few = min(1-fc,fw)
    TAE = 1000*(params.CC-0.5*params.PM)*params.ZE
    
    if t == 1: De_i = TAE - params.AFE
    else: De_i = max(De_f-p, 0)
        
    if De_i < params.AFE: kr = 1
    else: kr = max((TAE - De_i)/(TAE - params.AFE), 0)
            
    ke = min(kr*(kcx-Kcb), few*kcx)
    E = ke * ETo_PM
    
    if t == 1:
        Dp = max(p, 0)
        De_f = min(De_i - p+(E/few) + Dp, TAE)
    else:
        Dp = max(p-De_f, 0)
        De_f = min(De_f-p+(E/few) + Dp, TAE)
        
    Kc = Kcb + ke
    ETc = Kc * ETo_PM
    return ETc, ke, De_f, De_i


def evapotranspiracaoReal(params, ap, aKcb, aETc, aKe, aETo_PM, De_i, Dr_f, t):
    if t <= (params.L_INI + params.L_CRES + params.L_MID):
        Zr = ((aKcb - params.KCB_INI)/(params.KCB_MID - params.KCB_INI))*(params.ZRX - params.ZRN) + params.ZRN
    else:
        Zr = params.ZRN
    
    TAR = 1000.*(params.CC - params.PM)*Zr
    AFR = params.F*TAR
    
    if t == 1:
        Dr_i = min( max((TAR-AFR)-ap+aETc, 0), TAR)
        if Dr_i < AFR:
            ks = 1
        else:
            ks = max( (TAR-Dr_i)/(TAR-AFR), 0)

        Dp = max( ap-aETc-De_i, 0 )
        kcr = aKe + ks * aKcb
        ETR = kcr * aETo_PM
        Tr = ks * aKcb *aETo_PM
        Dr_f = max( min((TAR-AFR)-ap+ETR+Dp, TAR), 0 )
    else:
        Dr_i = min( max(Dr_f-ap-aETc, 0), TAR)
        if Dr_i < AFR:
            ks = 1
        else:
            ks = max( (TAR-Dr_i)/(TAR-AFR), 0)
        
        Dp = max( ap-aETc-Dr_f, 0)
        kcr = aKe + ks*aKcb
        ETR = kcr * aETo_PM
        Tr = ks*aKcb*aETo_PM
        Dr_f = min(Dr_f-ap+ETR+Dp, TAR)
    return  TAR, Dr_f

In [3]:
#Precisa instalar xlrd ("pip install xlrd" ou "conda install xlrd")
dados = pd.read_csv("datasets/Dados _Meteorologicos_2021a2022.csv", sep=';')
params = pd.read_csv("datasets/Parametros_DadosProcessados.csv", sep=';').squeeze()
resultados = pd.read_csv("datasets/DadosProcessados.csv", sep=';')
intermediarios = pd.read_csv("datasets/datas.csv", sep=';')
#tx = temperatura maxima
#tn = temperatura minima
#urx = umidade maxima
#urn = umidade minima
#u2 = vento
#qg = radiação
#p = precipitacao
#arm = 
#teta = 
#p = -
#irrigacao = 0
#exp = 0

In [4]:
intermediarios.tm

0      28.60
1      31.55
2      32.80
3      32.45
4      31.75
       ...  
116    28.35
117    28.55
118    26.95
119    28.35
120    27.90
Name: tm, Length: 121, dtype: float64

In [10]:
aARM = []
De_f = 0
De_i = 0
Dr_f = 0
#for idx, (doy, tx, tn, urx, urn, u2, qg, ap, _, _, _, _, _) in dados.iterrows():
for idx, (_, _, doy, _, tx, tn, urx, urn, ap, u2, _, qg, _) in dados.iterrows():
    
    # Termo Aerodinâmico    
    #tn = tx = intermediarios.tm[idx]
    S, tm, es, ea, lamb = termoAeroDinamico(params, urx, urn, tx, tn)
    print(ea, lamb, urx, urn, tx, tn, sep='\t')
    #print(isclose(intermediarios.S[idx], S), isclose(intermediarios.tm[idx], tm), isclose(intermediarios.es[idx], es), isclose(intermediarios.ea[idx], ea), isclose(intermediarios.lamb[idx], lamb), sep = '\t')
    
    Qn = saldoDeRadiacao(params, doy, qg, tx, tn, ea)
    
    #Evapotranspiração de referência [mm/d]
    aETo_PM = (0.408*S*(Qn)+lamb*900*u2*(es-ea)/(tm + 273))/(S+lamb*(1+0.34*u2))
    #print(aETo_PM)
    
    aKcb, kcx = transpiracaoPotencial(params, u2, urn, aETo_PM, idx+1)
    aETc, aKe, De_f, De_i = evaporacao(params, ap, aKcb, kcx, aETo_PM, De_f, idx+1)
    aTAR, Dr_f = evapotranspiracaoReal(params, ap, aKcb, aETc, aKe, aETo_PM, De_i, Dr_f,idx+1)
    
    aARM.append( max(aTAR - Dr_f,0) )

2.2220921799960243	0.06450643055036907	77.6	51.2	31.6	22.1
1.7190382094192376	0.06450643055036907	77.6	27.8	36.3	19.5
1.7988911402880823	0.06450643055036907	77.4	25.3	37.7	21.2
2.008131954244933	0.06450643055036907	77.5	33.1	37.0	21.1
1.9221751756924212	0.06450643055036907	77.5	31.2	36.5	21.1
1.9553923039711378	0.06450643055036907	77.6	34.5	35.7	20.7
2.0322761824720628	0.06450643055036907	77.7	35.1	36.2	21.2
1.895744651136112	0.06450643055036907	85.4	28.6	37.0	20.0
1.8047781379390309	0.06450643055036907	77.9	22.7	38.1	22.3
1.6128611906351182	0.06450643055036907	77.7	21.5	37.5	20.2
1.4581702723848269	0.06450643055036907	77.8	17.4	38.0	19.5
1.4496750107531216	0.06450643055036907	77.8	16.2	38.5	19.8
1.4391698625740026	0.06450643055036907	77.7	14.7	39.8	19.9
1.7655985742420375	0.06450643055036907	77.5	25.6	37.5	20.6
2.151080335640877	0.06450643055036907	77.5	35.9	37.5	21.5
2.032057973942921	0.06450643055036907	77.8	57.6	27.9	20.7
1.8010842225913464	0.06450643055036907	77.7	26.7	35.7	21.9
2

In [54]:
#with pd.set_option('display.max_columns', None):
resultados.head()

,ID,FK_PARAMETRO,CHAVE_IMPORTACAO,CHAVE_PROCESSAMENTO,COEFICIENTE_CULTURA,COEFICIENTE_BASAL,COEFICIENTE_EVAPO,EVAPO_CULTURA,TRANSP_POTENCIAL,EVAPO,ARMAZENAMENTO_AGUA,TAR,ARM2,EVAPO_REFERENCIA,EVAPO_REAL,TR,TA,DIAJULIANO,TARMEDIO,STRESS,EVAPO_ACUMULADA,ISNA,FK_USUARIO,DATANEW,IRRIGACAO_EFETIVA,IRRIGACAO_PROJETADA,PLUVIOSIDADE,DEFICIT,EXCEDENTE,FK_PROPRIEDADE,INDICE,DATA_IMPORTACAO,DATA
0,73229,40,a9af814529a859ca71d125e37c902ca3,NaN,1.385883,0.5,0.885883,14.311259,5.163229,9.14803,0.0,10.0,0.0,10.326457,9.14803,0.0,0.0,249,5.0,NaN,NaN,0.639219,10,06/09/2021,0.0,10.0,0.0,10.0,0.0,1,1,2022-09-20 14:30:00,NaN
1,73230,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,4.725286,4.725286,0.00000,0.0,10.0,0.0,9.450572,0.00000,0.0,0.0,250,5.0,NaN,NaN,0.000000,10,07/09/2021,0.0,10.0,0.0,10.0,0.0,1,2,2022-09-20 14:30:00,NaN
2,73231,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,4.926514,4.926514,0.00000,0.0,10.0,0.0,9.853029,0.00000,0.0,0.0,251,5.0,NaN,NaN,0.000000,10,08/09/2021,0.0,10.0,0.0,10.0,0.0,1,3,2022-09-20 14:30:00,NaN
3,73232,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,3.103985,3.103985,0.00000,0.0,10.0,0.0,6.207971,0.00000,0.0,0.0,252,5.0,NaN,NaN,0.000000,10,09/09/2021,0.0,10.0,0.0,10.0,0.0,1,4,2022-09-20 14:30:00,NaN
4,73233,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,4.754294,4.754294,0.00000,0.0,10.0,0.0,9.508587,0.00000,0.0,0.0,253,5.0,NaN,NaN,0.000000,10,10/09/2021,0.0,10.0,0.0,10.0,0.0,1,5,2022-09-20 14:30:00,NaN


In [52]:
aETo_PM

13.285857228495745